<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-09-15 10:21:52


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=d6211e0c512051c388ae5dad72c8157493ad2bd1ced147aa3774446906881b4a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [5]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','StarStock', 'BizCheck', 'FinCheck', 'TechCheck', 'Conviction', 'Category', 'Stock']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [6]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
0    70
1    67
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    29
V40     26
AR       5
V200     2
SS       1
VR       1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    125
0     12
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1    30
0    25
Name: StarStock, dtype: int64
-------


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-08-18 00:00:00+05:30,19310.0,18314.0,5.44,^NSEI,20150.0,43.0
2023-08-25 00:00:00+05:30,19266.0,18346.0,5.01,^NSEI,20150.0,41.0
2023-09-01 00:00:00+05:30,19435.0,18374.0,5.78,^NSEI,20150.0,51.0
2023-09-08 00:00:00+05:30,19820.0,18408.0,7.67,^NSEI,20150.0,67.0
2023-09-15 00:00:00+05:30,20150.0,18449.0,9.22,^NSEI,20150.0,76.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [12]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [13]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
#df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Max']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '200_SMA', 'Symbol', 'Target', 'Max', 'Close'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [14]:
# top 10 prospects to invest from BTT stocks
df_prospects[df_prospects['Strategy'] == 'BTT'].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
UJJIVANSFB.NS,45.74,56.0,1000.0,1000.0,1000.0,V40N,BTT,1,1,1,1,1.0,M,BANKS,14.0
ASIANPAINT.NS,4.32,39.0,65.9,35.2,27.7,V40,BTT,1,1,1,1,1.0,H,PAINTS,11.0
AKZOINDIA.NS,10.75,39.0,32.4,34.7,26.1,V40,BTT,1,1,1,1,NaN,M,NaN,10.0
ABBOTINDIA.NS,6.14,50.0,47.5,41.1,31.6,V40,BTT,1,1,1,1,NaN,M,PHARMA,6.0
BAJFINANCE.NS,13.85,64.0,36.7,11.8,23.5,V40,BTT,1,1,1,1,1.0,H,FINANCE,5.0
COLPAL.NS,20.18,52.0,124.0,180.0,158.0,V40,BTT,1,1,0,1,1.0,H,FMCG,4.0


In [15]:
# prospects to invest from star + V40/V40N/VR + below 200 DMA
df_prospects[(df_prospects['Criteria'].notnull()) & (df_prospects['StarStock'] == 1) & (df_prospects['Dev%_200'] < 5)].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
VINATIORGA.NS,0.50,57.0,44.3,30.4,22.60,V40N,NaN,1,1,1,NaN,NaN,M,NaN,23.0
RELIANCE.NS,-0.17,48.0,25.8,10.0,8.94,V40,ATH,1,1,1,1,1.0,H,REFINERIES,15.0
DABUR.NS,2.17,50.0,58.3,23.3,19.50,V40,NaN,1,1,0,1,1.0,H,FMCG,14.0
ASIANPAINT.NS,4.32,39.0,65.9,35.2,27.70,V40,BTT,1,1,1,1,1.0,H,PAINTS,11.0
HINDUNILVR.NS,-3.63,36.0,56.2,26.6,20.50,V40,NaN,1,1,0,NaN,1.0,H,FMCG,11.0


In [16]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
ADANIGREEN.NS,-11.80,53.0,128.00,7.81,22.80,NaN,NaN,1,1,0,1,0.0,L,NaN,197.0
BLUESTARCO.NS,11.81,60.0,52.80,24.50,23.40,AR,ATH,1,1,1,1,1.0,H,AC,90.0
TATAELXSI.NS,7.68,53.0,448.00,60.00,39.30,V40N,NaN,1,1,1,NaN,NaN,M,NaN,45.0
SBICARD.NS,2.66,46.0,35.40,13.00,25.70,NaN,NaN,1,1,0,NaN,1.0,M,NaN,36.0
KANSAINER.NS,14.22,50.0,49.40,14.50,10.70,V40N,NaN,1,1,1,NaN,NaN,M,NaN,32.0
MUTHOOTFIN.NS,17.04,55.0,13.70,12.10,17.90,NaN,NaN,1,1,0,NaN,0.0,L,FINANCE,27.0
MOTILALOFS.NS,22.83,45.0,8.91,13.00,15.60,V40N,NaN,1,1,0,NaN,0.0,L,FINANCE,27.0
TASTYBITE.NS,41.88,51.0,1000.00,1000.00,1000.00,NaN,NaN,1,1,1,NaN,NaN,M,NaN,24.0
VINATIORGA.NS,0.50,57.0,44.30,30.40,22.60,V40N,NaN,1,1,1,NaN,NaN,M,NaN,23.0


In [17]:
# top 10 prospects to invest from  V40/V40N/VR
df_prospects[df_prospects['Criteria'].notnull()].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
JCHAC.NS,10.76,53.0,1000.0,1000.00,1000.00,V40N,NaN,1,0,1,NaN,NaN,M,NaN,160.0
TANLA.NS,30.93,54.0,29.5,37.90,31.20,AR,ATH,0,0,1,1,0.0,M,IT,92.0
BLUESTARCO.NS,11.81,60.0,52.8,24.50,23.40,AR,ATH,1,1,1,1,1.0,H,AC,90.0
LALPATHLAB.NS,11.49,64.0,74.7,18.50,15.00,V40N,NaN,1,0,1,NaN,NaN,M,NaN,77.0
SFL.NS,-2.84,46.0,52.0,13.60,12.50,V40N,NaN,1,0,1,NaN,NaN,M,NaN,76.0
DREAMFOLKS.NS,0.75,34.0,35.9,76.90,60.20,AR,ATH,0,0,0,1,0.0,L,MISC,65.0
SYMPHONY.NS,-5.70,51.0,55.6,14.80,13.50,V40N,NaN,1,0,1,NaN,NaN,M,NaN,64.0
WHIRLPOOL.NS,15.07,58.0,99.2,8.78,6.37,V40,NaN,1,0,0,NaN,1.0,M,NaN,62.0
PFIZER.NS,-1.51,49.0,115.0,27.20,19.80,V40,NaN,1,0,0,NaN,0.0,L,PHARMA,55.0


In [18]:
# top 10 prospects to invest below 200 DMA
df_prospects[df_prospects['Dev%_200'] < 0].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
ADANIGREEN.NS,-11.80,53.0,128.0,7.81,22.80,NaN,NaN,1,1,0,1,0.0,L,NaN,197.0
ALKYLAMINE.NS,-3.06,51.0,65.9,41.30,41.40,NaN,NaN,0,0,1,1,0.0,M,CHEMICALS,88.0
SFL.NS,-2.84,46.0,52.0,13.60,12.50,V40N,NaN,1,0,1,NaN,NaN,M,NaN,76.0
SYMPHONY.NS,-5.70,51.0,55.6,14.80,13.50,V40N,NaN,1,0,1,NaN,NaN,M,NaN,64.0
PFIZER.NS,-1.51,49.0,115.0,27.20,19.80,V40,NaN,1,0,0,NaN,0.0,L,PHARMA,55.0
UPL.NS,-7.31,64.0,17.5,15.70,13.40,NaN,NaN,1,0,0,NaN,NaN,L,NaN,29.0
VEDL.NS,-8.79,45.0,11.9,23.80,20.40,NaN,NaN,1,0,0,NaN,NaN,L,NaN,24.0
RELIANCE.NS,-0.17,48.0,25.8,10.00,8.94,V40,ATH,1,1,1,1,1.0,H,REFINERIES,15.0
HINDUNILVR.NS,-3.63,36.0,56.2,26.60,20.50,V40,NaN,1,1,0,NaN,1.0,H,FMCG,11.0


In [19]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 12


,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
AXISBANK.NS,10.92,67.0,13.7,6.16,14.5,V40,NaN,1,1,0,1,1.0,M,BANKS,0.0
BAJAJ-AUTO.NS,24.72,76.0,22.0,26.20,20.2,V40,NaN,1,1,0,1,NaN,L,NaN,0.0
HCLTECH.NS,18.64,80.0,23.4,28.30,23.0,V40,NaN,1,1,0,NaN,1.0,H,IT,0.0
ICICIBANK.NS,9.61,64.0,18.6,6.35,17.2,V40,NaN,1,1,0,NaN,1.0,H,BANKS,0.0
MCDOWELL-N.NS,21.78,61.0,62.6,19.90,17.5,V40N,NaN,1,1,0,NaN,0.0,L,BREWERIES,1.0
OFSS.NS,28.81,67.0,20.7,35.10,24.8,V40N,NaN,1,1,1,NaN,1.0,M,IT,1.0
TITAN.NS,19.91,71.0,89.6,25.10,30.8,V40,NaN,1,1,0,NaN,1.0,H,JEWELLERY,1.0
MARICO.NS,11.38,57.0,55.3,43.00,35.8,V40,NaN,1,1,0,NaN,1.0,M,FMCG,2.0
ASTRAZEN.NS,27.89,70.0,68.6,30.90,23.0,V40N,NaN,1,1,1,1,NaN,M,NaN,3.0


In [20]:
# prospects for interactive analysis

data_table.DataTable(df_prospects, include_index=True)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
3MINDIA.NS,25.59,55.0,71.3,31.40,23.40,V40N,NaN,1,1,1,NaN,NaN,M,NaN,5.0
ABBOTINDIA.NS,6.14,50.0,47.5,41.10,31.60,V40,BTT,1,1,1,1,NaN,M,PHARMA,6.0
ACC.NS,4.41,57.0,45.3,10.00,7.06,NaN,NaN,0,0,0,1,0.0,L,CEMENT,33.0
ADANIENT.NS,1.32,53.0,104.0,10.10,9.63,NaN,NaN,1,0,0,1,1.0,M,NaN,64.0
ADANIGREEN.NS,-11.80,53.0,128.0,7.81,22.80,NaN,NaN,1,1,0,1,0.0,L,NaN,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VOLTAS.NS,9.91,66.0,89.9,9.74,4.42,NaN,NaN,1,0,0,NaN,0.0,L,AC,47.0
WHIRLPOOL.NS,15.07,58.0,99.2,8.78,6.37,V40,NaN,1,0,0,NaN,1.0,M,NaN,62.0
WSTCSTPAPR.NS,29.47,60.0,4.5,61.80,44.80,VR,ATH,0,0,0,NaN,NaN,L,PAPER,8.0


In [21]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [22]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-08-18 00:00:00+05:30,705.0,613.0,15.05,BERGEPAINT.NS,854.0,61.0
2023-08-25 00:00:00+05:30,704.0,616.0,14.34,BERGEPAINT.NS,854.0,57.0
2023-09-01 00:00:00+05:30,720.0,619.0,16.29,BERGEPAINT.NS,854.0,64.0
2023-09-08 00:00:00+05:30,717.0,621.0,15.44,BERGEPAINT.NS,854.0,58.0
2023-09-15 00:00:00+05:30,721.0,624.0,15.55,BERGEPAINT.NS,854.0,58.0
